# Rust model estimation

In [ ]:
import numpy as np
import os
import pandas as pd
import string as str
import math
import sys
import time

from scipy import optimize, special
import gurobipy as grb

from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML

In [ ]:
thePath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]

In [ ]:
n = 10
omax = 450000

In [ ]:
fileArr = ["g870.asc", "rt50.asc", "t8h203.asc", "a530875.asc", "a530874.asc", "a452374.asc", "a530872.asc", "a452372.asc"]
nbRowsArr = [36,60,81,128,137,137,137,137]
nbColsArr  = [15, 4,48,37,12,10,18,18]

In [ ]:
nbBuses = sum(nbColsArr)
nbMonths = max(nbRowsArr)-11
data = []
inmo = 12
inye = 74

In [ ]:
output = np.empty((nbBuses,nbMonths,3))
output[:]=np.nan

outputdiscr = np.empty((nbBuses,nbMonths,3))
outputdiscr[:] = np.nan

transitions = np.zeros((n,n))
transitions_list = []

In [ ]:
curbus = 0

for busType in range(len(fileArr)):

    thefile = fileArr[busType]
    nbRows = nbRowsArr[busType]
    nbCols = nbColsArr[busType]
    
    modeldata = np.genfromtxt(thePath + 'data_mec_optim\\dynamicchoice_Rust\datafiles\\' + thefile, delimiter='\n')
    modeldata = modeldata.reshape(nbCols, nbRows).T
    
    print("Group = ", busType, "; Nb at least one = ", len(np.nonzero(modeldata[5,:])[0]), "; Nb no repl = ", nbCols - len(np.nonzero(modeldata[5,:])[0]))
    
    for busId in range(nbCols):
        mo1stRepl = modeldata[3, busId]
        ye1stRepl = modeldata[4, busId]
        odo1stRepl = modeldata[5, busId]

        mo2ndRepl = modeldata[6, busId]
        ye2ndRepl = modeldata[7, busId]
        odo2ndRepl = modeldata[8, busId]

        moDataBegins = modeldata[9, busId]
        yeDataBegins = modeldata[10, busId]

        odoReadings = modeldata[11:nbRows, busId] 

        wasreplacedonce = np.asarray([1 if (x >= odo1stRepl) and (odo1stRepl > 0) else 0 for x in odoReadings])
        wasreplacedtwice = np.asarray([1 if (x >= odo2ndRepl) and (odo2ndRepl > 0) else 0 for x in odoReadings]) 
        howmanytimesreplaced = wasreplacedonce+wasreplacedtwice

        correctedmileage = odoReadings+np.multiply(howmanytimesreplaced,howmanytimesreplaced-2) * odo1stRepl - 0.5 * np.multiply(howmanytimesreplaced,howmanytimesreplaced-1) * odo1stRepl

        output[curbus, 0:(nbRows-12), 0] = howmanytimesreplaced[1:(nbRows-11)] - howmanytimesreplaced[0:(nbRows-12)]
        output[curbus, 0:(nbRows-12), 1] = correctedmileage[0:nbRows-12]
        output[curbus, 0:(nbRows-12), 2] = modeldata[12:nbRows, busId] - modeldata[11:nbRows-1, busId] 

        outputdiscr[curbus,:,0] = output[curbus,:,0] 
        outputdiscr[curbus,:,1:3] =  np.ceil(n*output[curbus,:,1:3]/omax)

        for t in range(nbRows-13):
            i = int(outputdiscr[curbus,t,1]-1)
            j = int(outputdiscr[curbus,t+1,1]-1)
            transitions[i,j]  = transitions[i,j] + 1

        curbus = curbus +1


In [ ]:
beta = 0.95

In [ ]:
Pi = (transitions.T/np.sum(transitions, axis = 1)).T
Pi = Pi[0:n-1,0:n-1]

In [ ]:
L = np.log(1-Pi[:,0])

In [ ]:
Pi_xx = Pi[:]
Pi_xx[1:(n-1),0] = 0
norm = np.sum(Pi_xx,axis = 1)
Pi_xx = (Pi_xx.T/norm).T

In [ ]:
U = (beta *Pi_xx - np.identity(n-1)).dot(L)